In [9]:
import sqlite3
conn = sqlite3.connect("AQI.db")
with conn:
    sql = '''
    CREATE TABLE IF NOT EXISTS records (
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        sitename TEXT NOT NULL,
        county TEXT,
        aqi INTEGER,
        status TEXT,
        pm25 NUMERIC,
        date TEXT,
        lat NUMERIC,
        lon NUMERIC,
        UNIQUE(sitename,date)
    );
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [10]:
import requests

url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    for items in data['records']:
        sitenames.add(items['sitename'])

    sitenames = list(sitenames)
print(sitenames)

['忠明', '新北(樹林)', '臺西', '屏東', '永和', '美濃', '安南', '線西', '花蓮', '臺南', '橋頭', '三義', '大寮', '前鎮', '沙鹿', '高雄（湖內）', '萬里', '三重', '關山', '觀音', '斗六', '松山', '大園', '左營', '萬華', '埔里', '臺南（麻豆）', '鳳山', '馬祖', '嘉義', '新店', '林口', '菜寮', '宜蘭（頭城）', '麥寮', '新港', '陽明', '臺東', '大同', '淡水', '中壢', '崙背', '桃園', '土城', '中山', '仁武', '西屯', '復興', '苗栗', '二林', '豐原', '馬公', '善化', '屏東（琉球）', '富貴角', '頭份', '彰化', '宜蘭', '新莊', '汐止', '潮州', '楠梓', '基隆', '金門', '大里', '竹山', '冬山', '彰化（員林）', '古亭', '屏東(枋山)', '平鎮', '士林', '小港', '大城', '林園', '新營', '前金', '朴子', '竹東', '板橋', '南投', '龍潭', '恆春', '湖口', '新竹']


In [11]:
import datasource
datasource.get_selected_data(sitename='屏東(枋山)')

[['2024-11-05 09:00', '屏東縣', 40, 3, '良好', 22.260899, 120.651472],
 ['2024-11-05 09:00', '屏東縣', 40, 3, '良好', 22.260899, 120.651472],
 ['2024-11-05 08:00', '屏東縣', 40, 3, '良好', 22.260899, 120.651472],
 ['2024-11-05 08:00', '屏東縣', 40, 3, '良好', 22.260899, 120.651472],
 ['2024-11-05 07:00', '屏東縣', 41, 43, '良好', 22.260899, 120.651472],
 ['2024-11-05 07:00', '屏東縣', 41, 43, '良好', 22.260899, 120.651472],
 ['2024-11-05 06:00', '屏東縣', 41, 3, '良好', 22.260899, 120.651472],
 ['2024-11-05 06:00', '屏東縣', 41, 3, '良好', 22.260899, 120.651472],
 ['2024-11-05 05:00', '屏東縣', 40, 2, '良好', 22.260899, 120.651472],
 ['2024-11-05 05:00', '屏東縣', 40, 2, '良好', 22.260899, 120.651472],
 ['2024-11-05 04:00', '屏東縣', 40, 2, '良好', 22.260899, 120.651472],
 ['2024-11-05 04:00', '屏東縣', 40, 2, '良好', 22.260899, 120.651472],
 ['2024-11-04 10:00', '屏東縣', 34, 18, '良好', 22.260899, 120.651472],
 ['2024-10-28 09:00', '屏東縣', 17, 6.2, '良好', 22.260899, 120.651472],
 ['2024-10-28 09:00', '屏東縣', 17, 6.2, '良好', 22.260899, 120.651472],
 ['

In [12]:
import sqlite3
conn = sqlite3.connect("AQI.db")
with conn:
    sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
    values ('屏東(枋山)','屏東縣', 34, '良好',18,'2024-11-04 10:00', 22.260899, 120.651472);
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [14]:
import sqlite3
conn = sqlite3.connect("AQI.db")
url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    with conn:
        cursor = conn.cursor()
        for items in data['records']:
            sitename = items['sitename']
            county = items['county']
            aqi = int(items['aqi']) if items['aqi'] != '' else 0
            status = items['status']
            pm25 = float(items['pm2.5']) if items['pm2.5'] != '' else 0.0
            date = items['datacreationdate']
            lon = float(items['longitude']) if items['longitude'] != '' else 0.0
            lat = float(items['latitude']) if items['latitude'] != '' else 0.0
            sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
                    values (?,?, ?, ?,?,?,?,?);
            '''
            cursor.execute(sql,(sitename, county, aqi, status,pm25,date,lat,lon))


In [15]:
conn = sqlite3.connect("AQI.db")
with conn:
    cursor = conn.cursor()
    sql = '''
    SELECT DISTINCT sitename
    FROM records
    '''
    cursor.execute(sql)
    sitenames = []

    for items in cursor.fetchall():
        sitenames.append(items[0])
    
    print(sitenames)

['屏東(枋山)', '宜蘭（頭城）', '新北(樹林)', '屏東（琉球）', '臺南（麻豆）', '高雄（湖內）', '彰化（員林）', '大城', '富貴角', '麥寮', '關山', '馬公', '金門', '馬祖', '埔里', '復興', '永和', '竹山', '中壢', '三重', '冬山', '宜蘭', '陽明', '花蓮', '臺東', '恆春', '潮州', '屏東', '小港', '前鎮', '前金', '左營', '楠梓', '林園', '大寮', '鳳山', '仁武', '橋頭', '美濃', '臺南', '安南', '善化', '新營', '嘉義', '臺西', '朴子', '新港', '崙背', '斗六', '南投', '二林', '線西', '彰化', '西屯', '忠明', '大里', '沙鹿', '豐原', '三義', '苗栗', '頭份', '新竹', '竹東', '湖口', '龍潭', '平鎮', '觀音', '大園', '桃園', '大同', '松山', '古亭', '萬華', '中山', '士林', '淡水', '林口', '菜寮', '新莊', '板橋', '土城', '新店', '萬里', '汐止', '基隆']


In [16]:
conn = sqlite3.connect("AQI.db")
with conn:
    cursor = conn.cursor()        
    sql = '''
    SELECT date,county,aqi,pm25,status,lat,lon
    FROM records
    WHERE sitename=?
    ORDER BY date DESC;
    '''
    cursor.execute(sql,('富貴角',))
    sitename_list = [list(item) for item in cursor.fetchall()]
    print(sitename_list)

[['2024-11-05 10:00', '新北市', 61, 22, '普通', 25.29681695, 121.53656894], ['2024-11-05 10:00', '新北市', 61, 22, '普通', 25.29681695, 121.53656894], ['2024-11-05 09:00', '新北市', 61, 17, '普通', 25.29681695, 121.53656894], ['2024-11-05 09:00', '新北市', 61, 17, '普通', 25.29681695, 121.53656894], ['2024-11-05 09:00', '新北市', 61, 17, '普通', 25.29681695, 121.53656894], ['2024-11-05 09:00', '新北市', 61, 17, '普通', 25.29681695, 121.53656894], ['2024-11-05 09:00', '新北市', 61, 17, '普通', 25.29681695, 121.53656894], ['2024-11-05 09:00', '新北市', 61, 17, '普通', 25.29681695, 121.53656894], ['2024-11-05 08:00', '新北市', 61, 18, '普通', 25.29681695, 121.53656894], ['2024-11-05 08:00', '新北市', 61, 18, '普通', 25.29681695, 121.53656894], ['2024-11-05 08:00', '新北市', 61, 18, '普通', 25.29681695, 121.53656894], ['2024-11-05 08:00', '新北市', 61, 18, '普通', 25.29681695, 121.53656894], ['2024-11-05 08:00', '新北市', 61, 18, '普通', 25.29681695, 121.53656894], ['2024-11-05 08:00', '新北市', 61, 18, '普通', 25.29681695, 121.53656894], ['2024-11-05 07:00'